In [4]:
%matplotlib inline

In [36]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.segmentation import clear_border
from skimage.measure import regionprops

# Get bubbles

In [41]:
filename = 'frame1.jpg'

In [42]:
CHANNEL_CODE = {'blue': 0, 'green': 1, 'red': 2}
DEFAULT_FILTERS = {'circularity_reciprocal': {'min': 0.2, 'max': 1.6},
                   'convexity': {'min': 0.92}}

channel = 'red'
raw_image = cv2.imread(filename)[:, :, CHANNEL_CODE[channel]]
current_image = raw_image.copy()


threshold = [20, 80]
dilate_footprint = 3
border_buffer_size = 3 
border_bgval = 1
erode_footprint = 1


current_image = cv2.Canny(current_image,
                 threshold[0],
                 threshold[1])


kernel = np.ones((dilate_footprint, dilate_footprint), np.uint8)
current_image = cv2.dilate(current_image, kernel, iterations=1)

h, w = current_image.shape[:2]  # stores image sizes
mask = np.zeros((h + 2, w + 2), np.uint8)
cv2.floodFill(current_image, mask, (0, 0), 0)

image_inv = cv2.bitwise_not(current_image)
current_image = clear_border(image_inv, buffer_size=border_buffer_size, bgval=border_bgval)

kernel = np.ones((erode_footprint, erode_footprint), np.uint8)
current_image = cv2.erode(current_image, kernel, iterations=1)

In [60]:
def calculate_circularity_reciprocal(perimeter, area):
    """calculate the circularity based on the perimeter and area"""
    return (perimeter**2)/(4*np.pi*area)


def calculate_convexity(perimeter, area):
    """calculate the circularity based on the perimeter and area"""
    return area/perimeter


def _bubble_properties_table(binary_image):
    """provide a label for each bubble in the image"""

    nbubbles, marker_image = cv2.connectedComponents(1 - binary_image)
    props = regionprops(marker_image)
    bubble_properties = \
        pd.DataFrame([{"label": bubble.label,
                       "area": bubble.area,
                       "centroid": bubble.centroid,
                       'xc': bubble.centroid[0],
                       'yc': bubble.centroid[1],
                       "convex_area": bubble.convex_area,
                       "equivalent_diameter": bubble.equivalent_diameter,
                       "perimeter": bubble.perimeter} for bubble in props])

    bubble_properties["convexity"] = \
        calculate_convexity(bubble_properties["perimeter"],
                            bubble_properties["area"])
    bubble_properties["circularity_reciprocal"] = \
        calculate_circularity_reciprocal(bubble_properties["perimeter"],
                                         bubble_properties["area"])

    bubble_properties = bubble_properties.set_index("label")

    return nbubbles, marker_image, bubble_properties


def _bubble_properties_filter(property_table, id_image,
                              rules=DEFAULT_FILTERS):
    """exclude bubbles based on a set of rules

    :return:
    """
    bubble_props = property_table.copy()
    all_ids = bubble_props.index.tolist()

    for prop_name, ruleset in rules.items():
        print(ruleset)
        for rule, value in ruleset.items():
            if rule == 'min':
                bubble_props = \
                    bubble_props[bubble_props[prop_name] > value]
            elif rule == 'max':
                bubble_props = \
                    bubble_props[bubble_props[prop_name] < value]
            else:
                raise Exception("Rule not supported, "
                                "use min or max as filter")

    removed_ids = [el for el in all_ids if el
                   not in bubble_props.index.tolist()]
    for idb in removed_ids:
        id_image[id_image == idb] = 0

    return id_image, bubble_props

def bubble_properties_calculate(binary_image,
                                rules=DEFAULT_FILTERS):
    """

    :param binary_image:
    :param rules:
    :return:
    """
    # get the bubble identifications and properties
    nbubbles, id_image, \
        prop_table = _bubble_properties_table(binary_image)
    # filter based on the defined rules
    id_image, properties = _bubble_properties_filter(prop_table,
                                                     id_image, rules)
    return id_image, properties

marker_image, props = bubble_properties_calculate(current_image)

{'min': 0.2, 'max': 1.6}
{'min': 0.92}


In [61]:
props

,area,centroid,xc,yc,convex_area,equivalent_diameter,perimeter,convexity,circularity_reciprocal
label,,,,,,,,,
5,7,"(4.857142857142857, 365.7142857142857)",4.857143,365.714286,7,2.985411,6.207107,1.127740,0.437996
12,9,"(8.0, 189.0)",8.000000,189.000000,9,3.385138,8.000000,1.125000,0.565884
16,31,"(8.903225806451612, 377.3225806451613)",8.903226,377.322581,33,6.282549,19.242641,1.611006,0.950512
21,6,"(9.833333333333334, 326.6666666666667)",9.833333,326.666667,9,2.763953,4.414214,1.359246,0.258432
26,19,"(10.578947368421053, 531.1052631578947)",10.578947,531.105263,31,4.918491,19.035534,0.998133,1.517633
...,...,...,...,...,...,...,...,...,...
2446,4,"(425.75, 324.25)",425.750000,324.250000,4,2.256758,3.207107,1.247230,0.204624
2447,8,"(425.125, 355.875)",425.125000,355.875000,8,3.191538,7.414214,1.079009,0.546802
2449,9,"(430.77777777777777, 630.1111111111111)",430.777778,630.111111,12,3.385138,8.414214,1.069619,0.626001


# Get speeds

In [88]:
frame_before = 'frame0.jpg'

prev_frame = cv2.imread(frame_before)
prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
current_frame = cv2.imread(filename)
current_frame = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

flow = cv2.calcOpticalFlowFarneback(prev_frame, current_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0).transpose(1, 0, 2)

In [89]:
props['yc'] = props['yc'].apply(lambda x: int(round(x, 0)))
props['xc'] = props['xc'].apply(lambda x: int(round(x, 0)))

In [84]:
flow.shape

(2, 800, 600)

In [79]:
raw_image.shape

(600, 800)

In [80]:
props['yc'].max()

774

In [81]:
props['xc'].max()

435

In [99]:
.shape

(864,)

In [102]:
flow.shape

(800, 600, 2)

In [105]:
flow[props['yc'].values].transpose(1, 0, 2)[props['xc'].values].shape

(864, 864, 2)

In [72]:
props['vy'] = flow[props['yc'], props['xc']]
props['vx'] = flow[]

,area,centroid,xc,yc,convex_area,equivalent_diameter,perimeter,convexity,circularity_reciprocal
label,,,,,,,,,
5,7,"(4.857142857142857, 365.7142857142857)",5,366,7,2.985411,6.207107,1.127740,0.437996
12,9,"(8.0, 189.0)",8,189,9,3.385138,8.000000,1.125000,0.565884
16,31,"(8.903225806451612, 377.3225806451613)",9,377,33,6.282549,19.242641,1.611006,0.950512
21,6,"(9.833333333333334, 326.6666666666667)",10,327,9,2.763953,4.414214,1.359246,0.258432
26,19,"(10.578947368421053, 531.1052631578947)",11,531,31,4.918491,19.035534,0.998133,1.517633
...,...,...,...,...,...,...,...,...,...
2446,4,"(425.75, 324.25)",426,324,4,2.256758,3.207107,1.247230,0.204624
2447,8,"(425.125, 355.875)",425,356,8,3.191538,7.414214,1.079009,0.546802
2449,9,"(430.77777777777777, 630.1111111111111)",431,630,12,3.385138,8.414214,1.069619,0.626001


In [ ]:
def draw_flow(im,flow,step=16):
    h,w = im.shape[:2]
#     y,x = mgrid[step/2:h:step,step/2:w:step].reshape(2,-1)
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int) 
    fx,fy = flow[y,x].T

    # create line endpoints
    lines = np.vstack([x,y,x+fx,y+fy]).T.reshape(-1,2,2)
    lines = np.int32(lines)

    # create image and draw
    vis = cv2.cvtColor(im,cv2.COLOR_GRAY2BGR)
    for (x1,y1),(x2,y2) in lines:
        cv2.line(vis,(x1,y1),(x2,y2),(0,255,0),1)
        cv2.circle(vis,(x1,y1),1,(0,255,0), -1)
    return vis


cap = cv2.VideoCapture(0)

ret,im = cap.read()
prev_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)

while True:
    # get grayscale image
    ret,im = cap.read()
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)

    # compute flow
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    prev_gray = gray
    
    
    # plot the flow vectors
    cv2.imshow('Optical flow',draw_flow(gray,flow))
    if cv2.waitKey(10) == 27:
        break